In [98]:
from manim import *
import math
import numpy as np
from manim_slides import *
import random
import os
os.environ["PATH"] = "/Library/TeX/texbin:" + os.environ["PATH"]

In [2]:
## Global Variables and Stuff ##
#config.media_embed = True
DELAY = 0.1

# latex preamble
texPre = TexTemplate()
texPre.add_to_preamble(r"""
    \usepackage{amsmath}
    \usepackage{amssymb}
    \newcommand{\E}{\mathbb{E}}
    \newcommand{\P}{\mathbb{P}}
    \newcommand{\vect}[1]{\mathbf{#1}}
""") 

# tex color dictionary
E_color = GREY_B
n_color = BLUE_C
T_color = ORANGE
F_color = PURPLE_A
k_color = GREEN
x_color = TEAL
f_color = r"#FFB347"
t2cD = {
    r"\mathbb{E}": E_color,
    r"\mathbb{P}": E_color,
    r"\big(": E_color,
    r"\big)": E_color,
    "n ": n_color,
    "T ": T_color,
    "F_": F_color,
    "k ": k_color,
    "x ": x_color,
    "x{}": x_color,
    "1{}": x_color,
    " f" : f_color
}

# font sizes
my_fs = 75

#other colors

house_color = WHITE
dot_color = WHITE
edge_color = BLUE

In [15]:

def stick_man(eye_angle = 0.1, my_scale = 0.5):
    # Head
    head = Circle(radius=0.7, fill_opacity=1,color=WHITE ).shift(UP * 0.5)
    # Draw the spider's eyes
    eye1 = Circle(radius=0.25, color=BLACK, fill_opacity=1).shift(UP * 0.7 + LEFT * 0.3)
    eye2 = Circle(radius=0.25, color=BLACK, fill_opacity=1).shift(UP * 0.7 + RIGHT * 0.3)
    # Add eyeballs to the eyes
    eye_x = np.cos(eye_angle)*0.1
    eye_y = np.sin(eye_angle)*0.1
    eyeball1 = Circle(radius=0.1, color=WHITE, fill_opacity=1).shift(UP * 0.7 + RIGHT*eye_x + UP*eye_y + LEFT * 0.3)
    eyeball2 = Circle(radius=0.1, color=WHITE, fill_opacity=1).shift(UP * 0.7 + RIGHT*eye_x + UP*eye_y + RIGHT * 0.3)
    # Body
    body = Line(head.get_bottom(), head.get_bottom() + DOWN * 1.5)
    # Arms (sloped downward from shoulders)
    shoulder_y = head.get_bottom()[1] - 0.3
    arms = VGroup(
        Line(start=[0, shoulder_y, 0], end=[-0.8, shoulder_y - 0.4, 0]),  # Left arm sloping down
        Line(start=[0, shoulder_y, 0], end=[0.8, shoulder_y - 0.4, 0])   # Right arm sloping down
    )
    # Legs (from bottom of body)
    leg_start = body.get_end()
    legs = VGroup(
        Line(leg_start, leg_start + DOWN * 1 + LEFT * 0.5),  # Left leg
        Line(leg_start, leg_start + DOWN * 1 + RIGHT * 0.5)  # Right leg
    )
    # Group and display
    return VGroup(head, eye1, eye2, eyeball1, eyeball2, body, arms, legs).scale(my_scale).set_z_index(2)

def get_angle(mobject1, mobject2):
    # Get direction vectors of the two Mobjects
    delta_x = mobject2.get_x() - mobject1.get_x()
    delta_y = mobject2.get_y() - mobject1.get_y()

    # Calculate the angle between the two vectors
    return math.atan2(delta_y, delta_x)
    

In [16]:
## Global Variables to Store Final Mobjects Passed Between Scenes ##
FINAL_MOBJECTS = None
FINAL_MOBJECTS_2 = None

In [106]:
%%manim -ql -v WARNING Test
#%%manim_slides -ql -v WARNING Spider --manim-slides controls=false
# Choose to do regular manim or manim-slides by choosing the correct jupyter magic. Must be first line.

class Test(Scene):  # Scene): # change to MyScene(Slide) for manim-slides
    def pause(self):
        self.wait(DELAY)
        #self.next_slide() #comment in for manim-slides
    
    def construct(self):
        # Create the number line using dots and connecting lines
        number_line_max = 4
        number_range = range(-number_line_max, number_line_max + 1)  # -2 to 2
        spacing = 1  # Horizontal spacing between dots

        dots = [Dot(point=[x * spacing, 0, 0],color=x_color) for x in number_range]
        lines = VGroup()
        line_color = GREY_A
        for i in range(len(dots) - 1):
            line = Line(dots[i].get_center(), dots[i + 1].get_center(),color=line_color).set_z_index(-1)
            lines.add(line)

        # Add ellipses on each end to suggest infinite extension
        x_buff = 0.6
        
        left_ellipsis = MathTex(r"\ldots",color=line_color).next_to(dots[0], LEFT, buff=x_buff)
        right_ellipsis = MathTex(r"\ldots",color=line_color).next_to(dots[-1], RIGHT, buff=x_buff)
        
        lines.add(Line(dots[0].get_center(), left_ellipsis.get_right() + RIGHT*x_buff*0.2,color=line_color).set_z_index(-1))
        lines.add(Line(dots[-1].get_center(), right_ellipsis.get_left() + LEFT*x_buff*0.2,color=line_color).set_z_index(-1))
        

        # Add labels below each dot
        labels = VGroup()
        y_buff = 0.4
        for i, x in enumerate(number_range):
            if x < 0:
                label = MathTex("-", str(abs(x)),color=x_color)
                target = dots[i].get_bottom() + DOWN * y_buff
                offset = target - label[1].get_center()
                label.shift(offset)
            else:
                label = MathTex(str(x),color=x_color)
                label.move_to(dots[i].get_bottom() + DOWN * y_buff)
            labels.add(label)

        # Assemble the number line
        number_line = VGroup(*dots, lines, labels, left_ellipsis, right_ellipsis)

        # Add the stick figure above the center dot (at 0)
        figure_pos = 0
        figure = stick_man().next_to(dots[number_line_max+figure_pos], UP, buff=0.1)  # dots[2] corresponds to x = 0

        #self.play(FadeIn(number_line), FadeIn(figure))
        self.add(number_line, figure)
        self.wait()

        #coin_flips = 'HTHH'
        n_flips = 6
        random.seed(1)
        coin_flips = ''.join(random.choice(['H', 'T']) for _ in range(n_flips))
        c_scale = 1.0
        coin_mobs = VGroup(*[H.copy().scale(c_scale) if c=='H' else T.copy().scale(c_scale) for c in coin_flips]).arrange(RIGHT)
        coin_mobs.to_corner(UL)
        
        for i in range(len(coin_mobs)):
            delta_x = (1 if coin_flips[i]=='H' else -1)
           
            flip_anims = animate_flip(coin_mobs[i],coin_flips[i])
            for ix in range(len(flip_anims)):
                #print(ix)
                flip_anim = flip_anims[ix]
                figure.generate_target()
                #print(figure_pos + (-1)**(ix%2))
                figure.target = stick_man(eye_angle = 3*PI/2 + PI/3*(-1)**(ix%2 + (delta_x+1)/2 ))
                 # get_angle(figure[0],))  #dots[number_line_max + figure_pos + 2*(-1)**(ix%2)]))
                figure.target.move_to(figure)
                self.play(flip_anim,MoveToTarget(figure),run_time=0.2)

            figure.generate_target()
            figure.target = stick_man(eye_angle = 3*PI/2)
            figure_pos_post = figure_pos + delta_x
            #print(figure_pos,figure_pos_post)
            figure.target.next_to(dots[number_line_max+figure_pos_post], UP, buff=0.1)
            self.play(MoveToTarget(figure,path_arc=(-1)**((delta_x+1)/2)*PI/2))
            figure_pos = figure_pos_post

        self.wait(2)


Manim Community v0.19.0

In [70]:
#Global variable with H coin and T coin used throughout
H = LabeledDot(MathTex(r"\stackrel{\text{H}}{\rightarrow}", color=BLACK), radius=0.35, color=BLUE).scale(1.5)
T = LabeledDot(MathTex(r"\stackrel{\text{T}}{\leftarrow}", color=BLACK), radius=0.35, color=YELLOW_B).scale(1.5)
empty_coin = LabeledDot(Tex("$T$", color=BLACK), radius=0.35, color=BLACK).scale(1.5)   


def set_target_location(A,B):
    A.target.set_x(B.get_x())
    A.target.set_y(B.get_y())
    
def set_new_location(A,B):
    A.set_x(B.get_x())
    A.set_y(B.get_y())


def animate_flip(coin,final='H',n_flips=1,side_H = None, side_T = None,my_scale=1):
    #RETURNS a list of animations that animate the mobject "coin" being flipped
    #The "final" variable incidicates what you want it to be at the end of the flipping
    #To animate a coin, use a loop to play the animations:
    
    # for a in animate_flip(coins[i],coin_flips[i]):
    #            self.play(a,run_time=0.2)
    
    global H,T
    
    if side_H == None:
        side_H = H
    
    if side_T == None:
        side_T = T
    
    full_fc = [side_H.copy().move_to(coin.get_center()),side_T.copy().move_to(coin.get_center())]
    anim_list = []
    for i in range(2):
        coin.generate_target()
        coin.target.stretch(0.01,dim=1)
        #coin.target.color = BLACK
        anim_list.append(MoveToTarget(coin))

        coin.color = BLACK
        coin.generate_target()

        offset = 1 if final=='H' else 0 #Ensures the coin lands on the side requested
        coin.target = full_fc[ (i+offset) % 2]

        anim_list.append(MoveToTarget(coin))
    return anim_list*n_flips

In [ ]:
%%manim -ql -v WARNING Test2
#%%manim_slides -ql -v WARNING Spider --manim-slides controls=false
# Choose to do regular manim or manim-slides by choosing the correct jupyter magic. Must be first line.

class Test2(Scene):  # Scene): # change to MyScene(Slide) for manim-slides
    def pause(self):
        self.wait(DELAY)
        #self.next_slide() #comment in for manim-slides
    def construct(self):
        #self.next_section()
        coin_flips = 'HTHH'
        c_scale = 1.0
        coin_mobs = VGroup(*[H.copy().scale(c_scale) if c=='H' else T.copy().scale(c_scale) for c in coin_flips]).arrange(RIGHT)
        
        for i in range(len(coin_mobs)):
            for a in animate_flip(coin_mobs[i],coin_flips[i]):
                self.play(a,run_time=0.2)
        self.wait(2)
    

Manim Community v0.19.0